In [2]:
%matplotlib inline
import os
import time 
import numpy as np
import pandas as pd
import matplotlib as mt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy, Precision,Recall, AUC
import tensorflow as tf

print(os.listdir('2000_images'))
tf.__version__

['test', 'train']


'2.8.0'

In [ ]:
WIDTH, HEIGHT, DEPTH, CHANNEL = 64,64,64,3

inputs = Input((WIDTH, HEIGHT, DEPTH, CHANNEL))

In [ ]:
#-----Arch.1--------

x = Conv3D(32, (3, 3, 3), padding='same', activation='relu')(inputs)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)

x=Flatten()(x)

#-----Arch.2--------

y = Conv3D(32, (3, 3, 3), padding='same', activation='relu')(inputs)
y = MaxPooling3D(pool_size=(2, 2, 2))(y)

y=Flatten()(y)

#---------Arch.3---------------

z = Conv3D(32, (3, 3, 3), padding='same', activation='relu')(inputs)
z = MaxPooling3D(pool_size=(2, 2, 2))(z)


z = Flatten()(z)

#------------------------------------------


fusion = concatenate([x,y])
fusion = concatenate([fusion,z])

fusion = Dense(1, activation='sigmoid')(fusion)



model = Model(inputs=inputs, outputs=(fusion))

METRICS = [Accuracy(),Precision(name="precision"), Recall(name="recall"),AUC(name="AUC")]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

model.summary()

In [ ]:
len(model.weights)

In [ ]:
# Train and Test the model
start= time.time()


train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory("../input/2000-images/2000 images/train",
                                                 target_size = (64,64,64),
                                                 batch_size = 3,
                                                 class_mode =  'binary')

test_set = test_datagen.flow_from_directory("../input/2000-images/2000 images/test",
                                            target_size = (64,64,64),
                                            batch_size = 3,
                                            class_mode =  'binary')

In [ ]:
model.fit(training_set,batch_size = 3, steps_per_epoch = 10, epochs = 30, validation_data=test_set)
print("Done in %.2f s." % (time.time() - start))

In [ ]:
imdim = (64,64,64,3)

enterimg = input("Enter the path of image ")   
test_image = image.load_img(enterimg)
print('-'*60)                                              

test_image = image.img_to_array(test_image).reshape((imdim[0],imdim[1],imdim[2],imdim[3]))
test_image = np.expand_dims(test_image, axis=0)


result = model.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Found'
    print('Lung Cancer Detected !')
else:
    print('Lung Cancer is not detected !')
    
print('-'*60)                                              



# display result
img = mpimg.imread(enterimg)
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(res.history.keys())

In [ ]:
# Analysis

# plot the loss
plt.plot(res.history['loss'] ,'g')
plt.plot(res.history['val_loss'] , 'b')
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss','Val loss'], loc='upper right')
plt.show()
plt.savefig('lossVal_loss')

# plot the accuracy
plt.plot(res.history['accuracy'])
plt.plot(res.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train acc','Val acc'], loc='upper right')
plt.show()
plt.savefig('AccuracyVal_acc')


print('\nTRAIN METRIC ----------------------')
print('Train acc: {}'.format(np.round((res.history['accuracy'][-1])*100, 2)))

print('\nTEST METRIC ----------------------')
print('Test acc: {}'.format(np.round((res.history['val_accuracy'][-1])*100, 2)))